In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import requests
import numpy as np
import pandas as pd

In [ ]:
MOORGATE = (51.518547, -0.088350)
DATE = "2014-12"

URL = "http://data.police.uk/api/crimes-street/all-crime?lat={}&lng={}&date={}".format(
    MOORGATE[0], MOORGATE[1], DATE
)

In [ ]:
response = requests.get(URL)
df = pd.read_json(response.content)
df.head()

In [ ]:
df.groupby('category')['context'].count().plot(kind='pie')

In [ ]:
# add an extra column!
extract_category = lambda x: x if x is None else x['category']
df['outcome_simplified'] = df['outcome_status'].apply(extract_category)

counter = df.groupby('outcome_simplified')['outcome_simplified'].count()
to_remove = counter[counter <= 10]
cleaned = counter.replace(to_remove, np.nan)

cleaned.plot(kind='pie')

In [ ]:
# extract the actual locations
df['location'][:4]
locations = [(float(l['latitude']), float(l['longitude'])) for l in df['location']]

In [ ]:
# visualize all the crimes on a map
from ipyleaflet import Map, Circle

def fill_map(limit=10):
    MOORGATE = (51.518547, -0.088350)
    m  = Map(center=MOORGATE, zoom=15)
    circles = [Circle(location=pos, radius=25) for pos in locations[:limit]]
    for c in circles:
        m.add_layer(c)
    
    return m

m = fill_map()
m

### Let's combine:
- __maps__ 
- and _interactive widgets_

In [ ]:
from ipywidgets import widgets, interact, FloatSlider, SelectMultiple, Dropdown, IntSlider

In [ ]:
@interact(x=FloatSlider(min=1e5, max=1e7, step=1e5))
def fun(x):
    print(x)

In [ ]:
@interact(category=Dropdown(options=list(df.category.unique()),
                            value=df.category.unique()[0],
                            description='Category:'),
         limit=IntSlider(10, min=1, max=100))
def draw_by_category(category='', limit=50):
    locations = [(float(l['latitude']), float(l['longitude'])) for l in df[df.category==category].location]
    circles = [Circle(location=pos, radius=30) for pos in locations[:limit]]
    
    for c in circles:
        m.add_layer(c)
    return m

draw_by_category()